# T1 motion correction tutorial

Start by importing the required libraries and defining some settings:

In [1]:
import os
import numpy as np
import nibabel as nib
import matplotlib.pyplot as plt

from ukat.data import fetch
from ukat.utils.tools import convert_to_pi_range
from ukat.mapping.t1 import T1, magnitude_correct
from ukat.moco.mdr import MotionCorrection

# Ensure figures are rendered in the notebook
%matplotlib inline

# Initialise output path for the Model-Driven Registration process
directory = 'T1_motion_correction_output'
os.makedirs(directory, exist_ok=True)
OUTPUT_DIR = os.path.join(os.getcwd(), directory)

In [2]:
# Fetch test data: fetch.t1_philips(1) is a single-slice acquisition
magnitude, phase, affine, ti, tss = fetch.t1_philips(1)

# Pre-processing as preparation for the Model-Driven Registration process
phase = convert_to_pi_range(phase)
complex_data = magnitude * (np.cos(phase) + 1j * np.sin(phase)) # convert magnitude and phase into complex data
ti = np.array(ti) * 1000  # convert TIs to ms
magnitude_corrected = np.nan_to_num(np.squeeze(magnitude_correct(complex_data)))
list_input_parameters = [affine, ti, 0, None, None, 2, False]

invalid value encountered in true_divide
invalid value encountered in true_divide
invalid value encountered in log


In [3]:
# Model-driven registration
registration = MotionCorrection(magnitude_corrected, affine, 'T1_Moco', list_input_parameters, convergence=3, multithread=False, log=True)
registration.run()

100%|██████████| 16384/16384 [02:20<00:00, 116.70it/s]
divide by zero encountered in true_divide
Co-registration progress:   0%|          | 0/19 [00:00<?, ?it/s]

Parameter Map: 
ParameterObject (000001DED919E110)
  RTTI typeinfo:   class elastix::ParameterObject
  Reference Count: 2
  Modified Time: 65
  Debug: Off
  Object Name: 
  Observers: 
    none
ParameterMap 0: 
  (ASGDParameterEstimationMethod "Original")
  (AutomaticParameterEstimation "true")
  (BSplineInterpolationOrder 1)
  (CheckNumberOfSamples "true")
  (DefaultPixelValue 0)
  (ErodeFixedMask "false")
  (ErodeMask "false")
  (FinalBSplineInterpolationOrder 1)
  (FinalGridSpacingInPhysicalUnits 50)
  (FixedImageDimension 2)
  (FixedImagePyramid "FixedSmoothingImagePyramid")
  (FixedInternalImagePixelType "float")
  (GridSpacingSchedule 2.80322 1.9881 1.41 1)
  (HowToCombineTransforms "Compose")
  (ImagePyramidSchedule 8)
  (ImageSampler "RandomCoordinate")
  (Interpolator "BSplineInterpolator")
  (MaximumNumberOfIterations 500)
  (MaximumNumberOfSamplingAttempts 8)
  (MaximumStepLength 0.1)
  (Metric "AdvancedMeanSquares")
  (Metric0Weight 1)
  (Metric1Weight 1)
  (MovingImageDime

RuntimeError: D:\a\im\_skbuild\win-amd64-3.7\cmake-build\_deps\elx-src\Core\Main\itkElastixRegistrationMethod.hxx:267:
ITK ERROR: ElastixRegistrationMethod(000001DED8A235E0): Internal elastix error: See elastix log (use LogToConsoleOn() or LogToFileOn()).

In [ ]:
# Get the coregistered input pixel_array
moco_array = registration.get_coregistered()
# Save motion corrected T1 sequence to NIfTI
registration.to_nifti(output_directory=OUTPUT_DIR, base_file_name='T1_motion_corrected', maps=['coregistered', 'fitted', 'deformation_field', 'params'])

In [ ]:
# Calculate maps from the original T1 sequence save as niftis
t1_mapper = T1(magnitude_corrected, ti, affine=affine, multithread=True, parameters=2)
t1_mapper.to_nifti(output_directory=OUTPUT_DIR, base_file_name='T1_original', maps='all')

# Calculate maps from the motion corrected T1 sequence and save as niftis
t1_moco_mapper = T1(moco_array, ti, affine=affine, multithread=True, parameters=2)
t1_moco_mapper.to_nifti(output_directory=OUTPUT_DIR, base_file_name='T1_moco', maps='all')

In [ ]:
fig, ax = plt.subplots(2, 1)

# Display the T1 map generated from the original data
im = ax.imshow(np.rot90(t1_mapper.t1_map[:, :, 2]), cmap='inferno', clim=(250, 2250))
cb = fig.colorbar(im, ax=ax[0, 0])
cb.set_label('$T_1 Original$ (ms)')
ax[0,0].axis('off')

# Display the T1 map generated from the motion corrected data
im = ax.imshow(np.rot90(t1_moco_mapper.t1_map[:, :, 2]), cmap='inferno', clim=(250, 2250))
cb = fig.colorbar(im, ax=ax[1, 0])
cb.set_label('$T_1 Moco$ (ms)')
ax[1,0].axis('off')

plt.show()